In [2]:
import os

In [3]:
%pwd

'd:\\vehicle_insurance_fraud_detection\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\vehicle_insurance_fraud_detection'

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path 

In [7]:
from src.vifd.constants import *
from src.vifd.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = CONFIG_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [18]:
import os
import urllib.request as request
import zipfile
from src.vifd import logger
from src.vifd.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-15 23:18:26,510: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-15 23:18:26,526: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-15 23:18:26,526: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-15 23:18:26,532: INFO: common: created directory at: artifacts]
[2023-12-15 23:18:26,532: INFO: common: created directory at: artifacts/data_ingestion]


[2023-12-15 23:18:28,256: INFO: 2353853782: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 68996
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e183ebd6a5b61282c4986b78d0fad9592ad3204a2291140e9b19adf0f43414c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C5A0:3EDF:58A480:6C0755:657C916A
Accept-Ranges: bytes
Date: Fri, 15 Dec 2023 17:48:27 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10238-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1702662507.644277,VS0,VE787
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 599b9b0ac4f7e148352aa43892a0cb8e77ce60d3
Expires: Fri, 15 Dec 2023 17:53:27 GMT
Source-Age: 0

]
